# three levels of logic in name recognition
1. filter single character
2. filtert the name contain unalphabetic characters
3. use Markov Chain model and Bigram to detect the valid name

Ref:
https://github.com/rrenaud/Gibberish-Detector

Using Dataset from 

https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-national-level-data

https://github.com/carltonnorthern/nickname-and-diminutive-names-lookup/blob/master/names.csv

https://catalog.data.gov/dataset/names-from-census-2000


# Markov Chain model training part, don't need to run

In [67]:
def normalize(line):
    """ Return only the subset of chars from accepted_chars.
    This helps keep the  model relatively small by ignoring punctuation, 
    infrequenty symbols, etc. """
    return [c.lower() for c in line if c.lower() in accepted_chars]

def ngram(n, l):
    """ Return all n grams from l after normalizing """
    filtered = normalize(l)
    for start in range(0, len(filtered) - n + 1):
        yield ''.join(filtered[start:start + n])
    
def avg_transition_prob(l, log_prob_mat):
    """ Return the average transition prob from l through log_prob_mat. """
    log_prob = 0.0
    transition_ct = 0
    n = 2
    for a, b in ngram(n, l):
        log_prob += log_prob_mat[pos[a]][pos[b]]
        transition_ct += 1
    # The exponentiation translates from log probs to probs.
    return math.exp(log_prob / (transition_ct or 1))

def train():
    """ Write a simple model as a pickle file """
    k = len(accepted_chars)
    # Assume we have seen 10 of each character pair.  This acts as a kind of
    # prior or smoothing factor.  This way, if we see a character transition
    # live that we've never observed in the past, we won't assume the entire
    # string has 0 probability.
    counts = [[10 for i in xrange(k)] for i in xrange(k)]

    # Count transitions from big text file, taken 
    # from http://norvig.com/spell-correct.html
    for line in open('name.txt'):
        for a, b in ngram(2, line):
            counts[pos[a]][pos[b]] += 1

    # Normalize the counts so that they become log probabilities.  
    # We use log probabilities rather than straight probabilities to avoid
    # numeric underflow issues with long texts.
    # This contains a justification:
    # http://squarecog.wordpress.com/2009/01/10/dealing-with-underflow-in-joint-probability-calculations/
    for i, row in enumerate(counts):
        s = float(sum(row))
        for j in xrange(len(row)):
            row[j] = math.log(row[j] / s)

    # Find the probability of generating a few arbitrarily choosen good and
    # bad phrases.
    good_probs = [avg_transition_prob(l, counts) for l in open('good.txt')]
    bad_probs = [avg_transition_prob(l, counts) for l in open('bad.txt')]

    # Assert that we actually are capable of detecting the junk.
    assert min(good_probs) > max(bad_probs)

    # And pick a threshold halfway between the worst good and best bad inputs.
    thresh = (min(good_probs) + max(bad_probs)) / 2
    pickle.dump({'mat': counts, 'thresh': thresh}, open('gib_model.pki', 'wb'))



if __name__ == '__main__':
    train()


# Please run from here 

In [30]:
import numpy as np
import math
import pickle
import pandas as pd

In [31]:
model_data = pickle.load(open('gib_model.pki', 'rb'))


In [89]:
accepted_chars = 'abcdefghijklmnopqrstuvwxyz '

pos = dict([(char, idx) for idx, char in enumerate(accepted_chars)])


def normalize(line):
    """ Return only the subset of chars from accepted_chars.
    This helps keep the  model relatively small by ignoring punctuation, 
    infrequenty symbols, etc. """
    return [c.lower() for c in line if c.lower() in accepted_chars]

def ngram(n, l):
    """ Return all n grams from l after normalizing """
    filtered = normalize(l)
    for start in range(0, len(filtered) - n + 1):
        yield ''.join(filtered[start:start + n])
    
def avg_transition_prob(l, log_prob_mat):
    """ Return the average transition prob from l through log_prob_mat. """
    log_prob = 0.0
    transition_ct = 0
    for a, b in ngram(2, l):
        log_prob += log_prob_mat[pos[a]][pos[b]]
        transition_ct += 1
    # The exponentiation translates from log probs to probs.
    return math.exp(log_prob / (transition_ct or 1))

def valid_name(string):
    if not string:
        return 0
    if len(string) < 2:
        return 0
    if not string.isalpha():
        return 0
    string = string.lower()
    
    # using the pre-trained threshold to return 1 or 0
    score = avg_transition_prob(string, model_data['mat'])
    return score
    if score >= model_data['thresh']:
        return 1
    else:
        return 0
        
        
    
    
    

In [34]:
valid_name('x')

0

In [35]:
valid_name('hg')

0

In [36]:
valid_name('fgg')

0

In [37]:
valid_name('gfsdgdfsg')

0

In [38]:
valid_name('gsdfgsf')

0

In [39]:
valid_name('fede')

1

In [40]:
valid_name('koli')

1

In [46]:
valid_name('Jose')

1

In [47]:
valid_name('Xinying')

1

In [48]:
valid_name('Sami')

1

In [49]:
valid_name('Haiyuan')

1

In [50]:
valid_name('Ankit')

1

In [53]:
valid_name('asdf')

0

In [56]:
valid_name('qazcds')

0

In [58]:
valid_name('sfffss')

0

In [60]:
valid_name('aaasdc')

0

In [61]:
valid_name('bcadc')

0

In [62]:
valid_name('pdded')

0

In [64]:
valid_name('ggghgh')

0

In [72]:
valid_name('saaas')

0

In [100]:
valid_name('eretytds')

0.020720264894560258